<a href="https://colab.research.google.com/github/PradipNichite/Youtube-Tutorials/blob/main/Youtube_NLP_Text_Representaion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install sklearn -q
# !pip install spacy -q
# !python -m spacy download en_core_web_sm

In [ ]:
import numpy as np
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
from sklearn import metrics

import string
import spacy
np.random.seed(42)

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Content Creation/Youtube Tutorials/datasets/toxic_commnets_500.csv",error_bad_lines=False, engine="python")
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)


,comment_text,toxic
0,Your knowledge of Hebrew? Thank god that Jossi...,0
1,==Tablighi Jamaat and allegations of terrorism...,0
2,"""\nWell it's understandable not wanting the in...",0
3,"""\n\n Please stop adding nonsense to Wikipedia...",0
4,"Saint Petersburg\nPlease, leave the sister cit...",0


In [ ]:
nlp = spacy.load("en_core_web_sm")
stop_words = nlp.Defaults.stop_words
print(stop_words)

{'is', 'besides', '‘m', 'go', 'regarding', 'whither', 'who', 'nowhere', 'any', 'whereby', '‘s', 'all', 'bottom', 'nevertheless', 'yourself', 'wherever', 'ourselves', 'most', 'nor', 'same', 'since', 'please', 'beforehand', 'back', 'may', 'then', 'make', 'however', 'because', 'becomes', 'on', 'thereafter', 'within', '‘d', '’re', 'was', 'have', 'her', 'alone', 'everyone', 'anyhow', 'towards', 'of', 'name', 'nothing', 'whom', 'once', 'therein', 'some', 'none', 'latter', 'mine', 'such', 'ever', 'never', 'you', 'themselves', 'their', '’d', 'see', 'meanwhile', 'others', 'are', 'not', 'hereafter', 'too', 'whose', 'n‘t', 'somewhere', 'them', 'him', 'that', 'hers', 'thereupon', 'were', 'up', "'m", 'whereupon', 'therefore', 'about', 'both', 'us', 'she', 'be', 'keep', 'been', 'mostly', 'someone', 'anyone', 'beyond', 'always', 'formerly', 'otherwise', 'fifteen', 'could', 'rather', 'down', 'yourselves', 'at', 'in', 'each', 'latterly', 'your', 'put', 'ca', 'what', 'already', 'thereby', 'unless', 'six

In [ ]:
punctuations = string.punctuation
print(punctuations)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [ ]:
# Creating our tokenizer function
def spacy_tokenizer(sentence):
    # Creating our token object, which is used to create documents with linguistic annotations.
    doc = nlp(sentence)



    # print(doc)
    # print(type(doc))

    # Lemmatizing each token and converting each token into lowercase
    mytokens = [ word.lemma_.lower().strip() for word in doc ]

    # print(mytokens)

    # Removing stop words
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # return preprocessed list of tokens
    return mytokens

In [ ]:
sentence = "I am eating apple ?"
spacy_tokenizer(sentence)

I am eating apple ?
<class 'spacy.tokens.doc.Doc'>
['i', 'be', 'eat', 'apple', '?']


['eat', 'apple']

In [ ]:
count_vector = CountVectorizer(tokenizer = spacy_tokenizer)
# tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)

In [ ]:
count_vector.fit_transform(["I am eating apple, I like apple","I am playing cricket"]).toarray() 

array([[1, 0, 1, 0],
       [0, 1, 0, 1]])

In [ ]:
count_vector.get_feature_names_out()

array(['apple', 'cricket', 'eat', 'play'], dtype=object)

In [ ]:
count_vector.vocabulary_

{'eat': 2, 'apple': 0, 'play': 3, 'cricket': 1}

In [ ]:
from sklearn.model_selection import train_test_split

X = data['comment_text'] # the features we want to analyze
ylabels = data['toxic'] # the labels, or answers, we want to test against

X_train, X_test, y_train, y_test = train_test_split(X, ylabels, test_size=0.2,stratify=ylabels)

In [ ]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression()

In [ ]:
X_train_vetcors= count_vector.fit_transform(X_train)
X_test_vetcors= count_vector.transform(X_test)

In [ ]:
X_train_vetcors.shape

(800, 5783)

In [ ]:
X_test_vetcors.shape

(200, 5783)

In [ ]:
X_train_vetcors.toarray()

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
classifier.fit(X_train_vetcors,y_train)

LogisticRegression()

In [ ]:
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.845
Logistic Regression Precision: 0.8709677419354839
Logistic Regression Recall: 0.81


In [ ]:
tfidf_vector = TfidfVectorizer(tokenizer = spacy_tokenizer)
X_train_vetcors= tfidf_vector.fit_transform(X_train)
X_test_vetcors= tfidf_vector.transform(X_test)

In [ ]:
classifier = LogisticRegression()
classifier.fit(X_train_vetcors,y_train)
predicted = classifier.predict(X_test_vetcors)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted))

Logistic Regression Accuracy: 0.86
Logistic Regression Precision: 0.9090909090909091
Logistic Regression Recall: 0.8
